<a id='top'><a/>
# Table of Contents
#### 1) Transforming time series data into de-trended, scaled and normalised format (i.i.d.) for tabular classification. 
    
   [Experiment 1.1 - grouping all companies as a whole for one transformation.](#expt1.1)
    
   [Pipeline and Hyperparameter Tuning](#pipe&Hyper)
    
   [Experiment 2.2 - individually transforming within companies across the same time-frame](#expt1.2)

#### 2) Use Statistics 
   [Experiment 2.1 - MANOVA](#expt2.1)
    
#### 3) Use Cluster Analysis 
   [Experiment 3.1 - k-means](#expt3.1)
    
#### 4) Use Sktime
   [Experiment 4.1 - ML Time Series Analysis](#expt4.1)

   [Appendix](#appendix)

# Introduction

## Data Preprocessing

In [1]:
# import dataframe and libraries
import pandas as pd
import numpy as np, array

from matplotlib import pyplot as plt
%matplotlib inline

# Function for creating model pipelines
from sklearn.pipeline import make_pipeline

# Import Logistic Regression
from sklearn.linear_model import LogisticRegression

# Import RandomForestClassifier and GradientBoostingClassifer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Function for splitting training and test set
from sklearn.model_selection import train_test_split

# Classification metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score

# GridSearchCV
from sklearn.model_selection import GridSearchCV
import seaborn as sns 
sns.set_style('darkgrid')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)

from IPython.display import display
from IPython.core import display as ICD

In [2]:
# Power Transform
from sklearn.preprocessing import power_transform

from scipy.stats import boxcox
from scipy.stats import yeojohnson

# Difference computation function
def difference(data, interval):
    return [data[i] - data[i - interval] for i in range(interval, len(data))]

# StandardScaler
from sklearn.preprocessing import StandardScaler

# Normalization
from sklearn.preprocessing import MinMaxScaler

In [3]:
main_df = pd.read_csv('abt.csv')
main_df.set_index(['Year', 'Quarter'], drop=True, inplace=True)
main_df.sort_index(ascending=True, sort_remaining= True, inplace=True)

In [4]:
main_df.groupby(main_df['company_name'], sort=True).sum()

,Accounts Receivable,Inventory,Total current assets,Total current liabilities,Total liabilities,Common Stock,Total liabilities and equity,New Deferred Revenue,New Property and Equipment,Total Non-Current Assets,Total_Assets,Non-Current Liabilities,Total Shareholder's Equity,Accounts Payables,Retained Earning,OCI,Accounts_Receivable_missing,Common_Stock_missing,Short_term_investments,Short_term_investments_missing,Inventory_missing,current_ratio,quick_ratio,debt_ratio,debt_to_equity_ratio,equity_multiplier,industry
company_name,,,,,,,,,,,,,,,,,,,,,,,,,,,
Amazon,2.829430e+11,3.825570e+11,1.428235e+12,1.301270e+12,2.219730e+12,2.410000e+08,3044300000000,1.271290e+11,1.046805e+12,2.108220e+11,3.044300e+12,9.184600e+11,8.245700e+11,7.327650e+11,2.964710e+11,2.222100e+10,0,0,2.756270e+11,0,0,59.065070,42.640448,34.963018,130.548865,180.548865,50.0
Apple,3.975220e+11,8.578300e+10,2.800189e+12,2.125556e+12,5.016973e+12,8.655780e+11,8110972000000,2.627400e+11,7.421370e+11,1.740391e+12,8.110972e+12,2.184731e+12,3.093999e+12,9.117180e+11,2.245244e+12,3.245900e+10,0,0,3.774262e+12,7,0,34.278200,33.245377,15.750090,43.698300,69.698300,26.0
Facebook,1.021680e+11,0.000000e+00,8.812530e+11,1.201380e+11,2.715360e+11,0.000000e+00,1752395000000,2.474000e+09,3.430340e+11,8.617100e+10,1.752395e+12,1.097640e+11,2.789311e+12,9.619000e+09,6.261750e+11,1.067900e+10,0,22,5.449710e+11,0,22,209.655930,209.655930,3.006712,1.913864,13.727506,22.0
Ford,4.271477e+12,3.004110e+11,3.144905e+12,2.622717e+12,6.070087e+12,1.188000e+09,6941199000000,1.209010e+12,9.493500e+11,3.796294e+12,6.941199e+12,1.301248e+12,8.665520e+11,6.549910e+11,6.151340e+11,3.666120e+11,0,0,6.122080e+11,0,0,35.919542,32.394452,26.276471,217.926688,248.112358,0.0
GM,1.644996e+12,5.700550e+11,3.200108e+12,2.963690e+12,6.241944e+12,5.930000e+08,7959308000000,1.268860e+12,2.115283e+12,4.846485e+12,7.959308e+12,3.278254e+12,1.627338e+12,1.051662e+12,6.460360e+11,2.701340e+11,0,2,3.161520e+11,3,0,47.476217,38.794746,33.521496,168.980508,214.518253,0.0
Google,3.111810e+11,1.197800e+10,2.279091e+12,5.004500e+11,8.594560e+11,7.801800e+11,3849115000000,3.170500e+10,9.026310e+11,5.296800e+10,3.849115e+12,6.940200e+10,3.010023e+12,5.818300e+10,2.257396e+12,2.755300e+10,0,0,1.583755e+12,0,4,93.089222,92.671373,4.140259,5.295098,24.123270,19.0
Microsoft,5.313600e+11,7.394500e+10,4.296721e+12,1.558512e+12,3.847046e+12,2.248386e+12,6524922000000,8.558170e+11,6.628030e+11,3.780890e+11,6.524922e+12,1.406331e+12,2.677876e+12,2.150510e+11,3.924650e+11,5.902300e+10,0,0,3.214832e+12,0,0,88.989467,87.415692,18.176794,46.010661,78.010661,32.0
Netflix,0.000000e+00,8.930012e+13,4.731783e+13,3.558797e+13,9.812288e+13,9.938387e+12,124929670160000,4.226629e+12,2.671635e+12,5.674708e+12,1.249297e+14,2.205699e+12,2.680679e+13,2.984036e+12,1.438237e+13,1.688469e+11,48,0,3.380962e+12,10,1,66.778027,-5.657607,35.801000,158.649349,206.649349,48.0
Tesla,1.344071e+10,5.293356e+10,1.465474e+11,1.450426e+11,3.730633e+11,2.932000e+06,477577304000,3.292231e+10,1.611449e+11,1.146351e+10,4.775773e+11,1.749331e+11,8.476468e+10,4.905543e+10,8.901459e+10,9.630870e+08,0,3,1.792980e+08,14,0,23.363573,14.832870,17.539468,108.706417,134.864269,0.0


In [5]:
#  Delete columns that will not be needed in Experiment 1.1.
main_df_1_1 = main_df.copy()
main_df_1_1.drop('Filing/Acc.No.', axis=1, inplace=True)
main_df_1_1.drop('company_name', axis=1, inplace=True)

In [6]:
#  Delete columns that will not be needed in Experiment 1.2.
main_df_1_2 = main_df.copy()
main_df_1_2.drop('Filing/Acc.No.', axis=1, inplace=True)

In [ ]:
# print(main_df_1_1.dtypes)
# print(main_df_1_1.shape)

A comparison of results between Scikit-Learn built-in preprocessing methods and Brown's transformation on Tesla_dataframe is provided in the appendix. Scikit-Learns methodology proved to 

#### Normalization
Normality need not to be assumed, let's see how the model performs first. We will try this if the unknown pattern cannot be mapped under the current parameter settings.

Decided to use Brown's method.

Inversion would not make sense in our case since we are not predicting variables as assumed in the Brown article, and we are not comparing a model performance (predictive or classifier or otherwise). 

***Here transforming acheives only one goal to standardise our different companies, so that reported financials can be compared directly accross companies.***

Brown Article on Time-Series Data Transformation for Machine Learning - https://machinelearningmastery.com/remove-trends-seasonality-difference-transform-python/#:~:text=Stationary%20Time%20Series,-The%20observations%20in&text=Time%20series%20are%20stationary%20if,the%20variance%20of%20the%20observations.

In [ ]:
def brown_transform(col_name, s, window):
    # Create a list to hold parameter information before transformation.
    para_list = []
    
    # Each Series -> variable s
    s = df[col_name]

    # Power Transform
    try:
        result, lmbda = boxcox(s)
#         print('box_cox')
#         print(lmbda)
        para_list.append(['B-C', lmbda])
        
    except ValueError:
        result, lmbda = yeojohnson(s.values.reshape(-1,1))
#         print('Yeo-Johnson')
        
        if type(lmbda) == np.float64:
            para_list.append(['Y-J', lmbda])
            
        else:
    #         print(f'lambda {lmbda} {type(lmbda)}')
            lmbda = lmbda[0]
            para_list.append(['Y-J', lmbda])
        
    # Difference Transform
    diff_res = difference(result, window)

    # StandardScaler
    # Turn 1-D array of lists into a 2-D array 
    diff_res = array(diff_res).reshape(len(diff_res), 1)
    
    para_list.append(np.mean(diff_res))
    para_list.append(np.std(diff_res))
    
    standard_res = StandardScaler().fit_transform(diff_res)

    # Normalization
#     para_list.append(min(standard_res)[0])
#     para_list.append(max(standard_res)[0])
    
    norm_res = MinMaxScaler(feature_range=(1,2)).fit_transform(standard_res)
    norm_res = [ s[0] for s in norm_res]
    trans_para_.update({ col_name : para_list} )
    
    return norm_res

The above preprocessing steps do not require a GridSearch to optimise parameter tuning. 

**Power Transform:** Boxcox() automates the lambda parameter by maximising  the log likelihood function for non-negative power transformations.

Since our data contains negative, zero-values, and postive values. BoxCox limitations in negative numerals is handled by Y-J; while zero values are handled by BoxCox which cannot be handled by Y-J.

**Difference Transform** Utilises the value difference between t=0 and t+=interval to smooth out trends and seasonality. Since our data is quarterly based, interval = 4.

**StandardScaler** Uses the mean and sd to normalise the data with zero mean and unit variance. 

**Normalization** Scales the data to fit with a predefined range (usually 0-1) for better comparison. 

Therefore, parameter tuning for preprocessing steps is unnecessary.

In order to prevent data leakage we will split the training and testing data first and apply training preprocessing parameters to the test set. 

### Isolate Train-model-data (non-Tesla Data) from data of interest (Tesla Data).

In [ ]:
tesla_df_1_1 = main_df_1_1[main_df_1_1['industry'].isna()]
tesla_df_1_1.shape

#### Train_df

In [ ]:
Train_df_1_1 = main_df_1_1[main_df_1_1['industry'].notna()]
Train_df_1_1.shape

#### Train_df

In [ ]:
# Create separate object for target variable
y = Train_df_1_1.industry

# Create separate object for input features
X = Train_df_1_1.drop('industry', axis=1)

In [ ]:
# Split X and y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=1234,
                                                    stratify=Train_df_1_1.industry)

# Print number of observations in X_train, X_test, y_train, and y_test
print( len(X_train), len(X_test), len(y_train), len(y_test) )

In [ ]:
X_test.sort_index(ascending=True, sort_remaining= True, inplace=True)
X_test.shape

In [ ]:
X_train.sort_index(ascending=True, sort_remaining= True, inplace=True)
X_train.shape

<a id='expt1.1'></a>

# Experiment 1.1 - grouping all companies as a whole for one transformation
<a href='#top'>Top of Page</a>

### 1) Transforming Tech Industry training_dfs

In [ ]:
df = X_train
# Create dictionary to save the transformation parameters.
trans_para_ = {}

transformed_df_list = []

# Apply to all training columns. 
for col_name in df.columns:
        
    # Apply only to numeric columns. 
    if df[col_name].dtypes == 'float64':
            
        s = df[col_name]
        
        # Brown's Transformation Suggestions
        result = brown_transform(col_name, s, 4)
        
        transformed_df = pd.DataFrame({col_name:result}, index=df.index[4:] )
        
        transformed_df.plot(kind='line')
        transformed_df_list.append(transformed_df)
        
        print(col_name)
        plt.show()
        

In [ ]:
new_X_train = pd.concat(transformed_df_list, axis=1)

### Implement the training transformations onto X_test with trans_para_.

In [ ]:
# X_test_new = scaler.transform(X_test)
X_test_new_list = []
for col, parameters in trans_para_.items():
    
    power_T = parameters[0][0]
    
    lmbda = parameters[0][1]
    
    mean_ = parameters[1]
    
    std_ = parameters[2]
    
    s = X_test[col]
#     s.plot(title=col)
#     plt.show()
    
    # Power Transformation
    if power_T == 'B-C':
        result = boxcox(s, lmbda)
        
    elif power_T == 'Y-J':
        result = yeojohnson(s, lmbda)
    
    # Difference Transformation
    diff_res = difference(result, 4)
    
    # StandardScaler 
    diff_res = array(diff_res).reshape(len(diff_res), 1)
    standard_res = StandardScaler(with_mean = mean_, 
                                 with_std = std_ ).fit_transform(diff_res)
    
    # Normalization
    norm_res = MinMaxScaler(feature_range=(1,2)).fit_transform(standard_res)
    norm_res = [ s[0] for s in norm_res]
    
    df = pd.DataFrame(dict({col: norm_res}), index=s.index[4:])
    X_test_new_list.append(df)
    
#     X_test_new.plot(title=col)
#     plt.show()
    

In [ ]:
new_X_Test = pd.concat(X_test_new_list, axis=1)
new_X_Test.head()

Below is a visual illustration of what happens when we transform our data using brown_transform(). 

<img src='before_and_after_transformation.png'>

<a id='pipe&Hyper'></a>
### Pipeline and Hyperparameters
<a href='#top'>Top of Page</a>

[Expt 2.2 - Train Model](#expt2.2train)

In [28]:
# Pipeline dictionary Classifiers
pipelines = {
    'l1' : make_pipeline(LogisticRegression(penalty='l1', random_state=123)),
    'l2' : make_pipeline(LogisticRegression(penalty='l2', random_state=123)),
    'rf' : make_pipeline(RandomForestClassifier(random_state=123)),
    'gb' : make_pipeline(GradientBoostingClassifier(random_state=123))
}

In [29]:
# Logistic Regression hyperparameters
l1_hyperparameters = {
    'logisticregression__C' : [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000],
}

l2_hyperparameters = {
    'logisticregression__C' : [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000],
}

# Random Forest hyperparameters
rf_hyperparameters = {
    'randomforestclassifier__n_estimators': [100, 200],
    'randomforestclassifier__max_features': ['auto', 'sqrt', 0.33],
    'randomforestclassifier__min_samples_leaf': [1, 3, 5, 10]
}

# Boosted Tree hyperparameters
gb_hyperparameters = {
    'gradientboostingclassifier__n_estimators': [100, 200],
    'gradientboostingclassifier__learning_rate': [0.05, 0.1, 0.2],
    'gradientboostingclassifier__max_depth': [1, 3, 5]
}

In [30]:
# Create hyperparameters dictionary
hyperparameters = {
    'l1' : l1_hyperparameters,
    'l2' : l2_hyperparameters,
    'rf' : rf_hyperparameters,
    'gb' : gb_hyperparameters
}

## Fit and Tune Model with Cross-Validation

In [ ]:
# Create empty dictionary called fitted_models
fitted_models = {}

# Loop through model pipelines, tuning each one and saving it to fitted_models
for name, pipeline in pipelines.items():
    # Create cross-validation object from pipeline and hyperparameters
    model = GridSearchCV(pipeline, hyperparameters[name], cv=10, n_jobs=-1)
    
    # Fit model on X_train, y_train
    model.fit(new_X_train, y_train[4:])
    
    # Store model in fitted_models[name] 
    fitted_models[name] = model
    
    # Print '{name} has been fitted'
    print(name, 'has been fitted.')

In [ ]:
for name, model in fitted_models.items():
    pred = model.predict_proba(new_X_Test)
    pred = [p[1] for p in pred]
    
    print( name, roc_auc_score(y_test[4:], pred) )

Plot the highest AUROC score.

In [ ]:
# Predict PROBABILITIES using L1-regularized logistic regression
pred = fitted_models['l2'].predict_proba(new_X_Test)

# Get just the prediction for the positive class (1)
pred = [p[1] for p in pred]

# Display first 10 predictions
print( np.round(pred[:10], 2) )

In [ ]:
# Calculate ROC curve from y_test and pred
fpr, tpr, thresholds = roc_curve(y_test[4:], pred)

In [ ]:
# Initialize figure
fig = plt.figure(figsize=(9,9))
plt.title('Receiver Operating Characteristic')

# Plot ROC curve
plt.plot(fpr, tpr, label='l1')
plt.legend(loc='lower right')

# Diagonal 45 degree line
plt.plot([0,1],[0,1],'k--')

# Axes limits and labels
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

The above shows the classification where tech is the positive class.

Now reversing the positive class from the Tech industry to Automobile. 


In [ ]:
def swapping_targets(y):
    swapped_y_list = []

    for value in y.values:

        if value == 1:
            swapped_y_list.append(0)

        if value == 0:
            swapped_y_list.append(1)
            
    swapped_y = pd.Series(swapped_y_list, index=y.index)
    
    return swapped_y

In [ ]:
new_y_train = swapping_targets(y_train)

In [ ]:
new_y_test = swapping_targets(y_test)

In [ ]:
# Create empty dictionary called fitted_models
fitted_models_2 = {}

# Loop through model pipelines, tuning each one and saving it to fitted_models
for name, pipeline in pipelines.items():
    # Create cross-validation object from pipeline and hyperparameters
    model = GridSearchCV(pipeline, hyperparameters[name], cv=10, n_jobs=-1)
    
    # Fit model on X_train, y_train
    model.fit(new_X_train, new_y_train[4:])
    
    # Store model in fitted_models[name] 
    fitted_models_2[name] = model
    
    # Print '{name} has been fitted'
    print(name, 'has been fitted.')

In [ ]:
for name, model in fitted_models_2.items():
    pred = model.predict_proba(new_X_Test)
    pred = [p[1] for p in pred]
    
    print( name, roc_auc_score(new_y_test[4:], pred) )

Swapping 0 and 1 produces the same results except for l1. 

In [ ]:
# Predict PROBABILITIES using L1-regularized logistic regression
pred = fitted_models_2['l2'].predict_proba(new_X_Test)

# Get just the prediction for the positive class (1)
pred = [p[1] for p in pred]

# Display first 10 predictions
print( np.round(pred[:10], 2) )

In [ ]:
# Calculate ROC curve from y_test and pred
fpr, tpr, thresholds = roc_curve(new_y_test[4:], pred)

In [ ]:
# Initialize figure
fig = plt.figure(figsize=(9,9))
plt.title('Receiver Operating Characteristic')

# Plot ROC curve
plt.plot(fpr, tpr, label='l1')
plt.legend(loc='lower right')

# Diagonal 45 degree line
plt.plot([0,1],[0,1],'k--')

# Axes limits and labels
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

## Expt 1.1 Conclusion
We will not try classifying Tesla into the models under these preprocessing conditions, because the predictability of the model is as good as chance.


<a id='expt1.2'></a>

# Experiment 1.2 - individually transforming within companies across the same time-frame
<a href='#top'>Top of Page</a>

In [7]:
tesla_df_1_2 = main_df_1_2[main_df_1_2['industry'].isna()]
Train_df_1_2 = main_df_1_2[main_df_1_2['industry'].notna()]

In [8]:
# Create separate object for target variable
y = Train_df_1_2.industry

# Create separate object for input features
X = Train_df_1_2.drop('industry', axis=1)

In [9]:
# Split X and y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=1234,
                                                    stratify=Train_df_1_2.industry)

# Print number of observations in X_train, X_test, y_train, and y_test
print( len(X_train), len(X_test), len(y_train), len(y_test) )

216 54 216 54


In [10]:
X_test.sort_index(ascending=True, sort_remaining= True, inplace=True)
X_train.sort_index(ascending=True, sort_remaining= True, inplace=True)

C:\Users\Matthew\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Matthew\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
company_list = [name for name in X_train.company_name.unique()]

In [13]:
df_lst = []
for name in company_list:
    company_df = X_train[X_train['company_name']==name]
    df_lst.append(company_df)

In [14]:
company_dfs = dict(zip(company_list, df_lst))

In [15]:
for name, df in company_dfs.items():
    print(name)
    partial_df = df.head(2)
    print(df.shape)
#     ICD.display(partial_df)

Amazon
(39, 27)
Netflix
(39, 27)
GM
(35, 27)
Microsoft
(28, 27)
Ford
(23, 27)
Apple
(20, 27)
Facebook
(16, 27)
Google
(16, 27)


new_brown_transform without difference transformation because too little data, seperating into company_df's drastically scarificed more data, expecially in X_test. 

In [16]:
def new_brown_transform(col_name, s):
    # Create a list to hold parameter information before transformation.
    para_list = []
    
    # Each Series -> variable s
    s = df[col_name]

    # Power Transform
    try:
        result, lmbda = boxcox(s)
#         print('box_cox')
#         print(lmbda)
        para_list.append(['B-C', lmbda])
        
    except ValueError:
        result, lmbda = yeojohnson(s.values.reshape(-1,1))
#         print('Yeo-Johnson')
        
        if type(lmbda) == np.float64:
            para_list.append(['Y-J', lmbda])
            
        else:
    #         print(f'lambda {lmbda} {type(lmbda)}')
            lmbda = lmbda[0]
            para_list.append(['Y-J', lmbda])
        
#     # Difference Transform
#     diff_res = difference(result, window)

    # StandardScaler
    # Turn 1-D array of lists into a 2-D array 
    #diff_res = array(diff_res).reshape(len(diff_res), 1)
    
    para_list.append(np.mean(result))
    para_list.append(np.std(result))
    
    result = np.array(result).reshape(len(result), 1)
    standard_res = StandardScaler().fit_transform(result)

    # Normalization
#     para_list.append(min(standard_res)[0])
#     para_list.append(max(standard_res)[0])
    
    norm_res = MinMaxScaler(feature_range=(1,2)).fit_transform(standard_res)
    norm_res = [ s[0] for s in norm_res]
    trans_para_.update({ col_name : para_list} )
    
    return norm_res

 **Perform columnar transformation on each company-dataframe.**

In [17]:
# { Company Name : {feat_name : [parameters]} }
company_trans_para = {}

trfm_com_df_lst = []

# For each company
for name, df in company_dfs.items():
    print(name)
    # Create dictionary to save the transformation parameters.
    trans_para_ = {}

    transformed_df_list = []

    # Apply to all training columns within that company.
    for col_name in df.columns:

        # Apply only to numeric columns. 
        if df[col_name].dtypes == 'float64':

            s = df[col_name]

            # Brown's Transformation Suggestions
            result = new_brown_transform(col_name, s)
            
            # Transformed Series 
            transformed_df = pd.DataFrame({col_name:result}, index=df.index )
            transformed_df_list.append(transformed_df)
#             transformed_df.plot(kind='line')
#             print(col_name)
#             plt.show()
            
    # Transformed Dataframe, gluing series tgt
    trans_df = pd.concat(transformed_df_list, axis=1)
    
    # Append transformed df to df_list
    trfm_com_df_lst.append(trans_df)
    
    # Saving transformation parameters. 
    company_trans_para[name] = trans_para_
    
#     if name == 'GM':
#         break

Amazon
Netflix
GM
Microsoft
Ford


C:\Users\Matthew\Anaconda3\lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\Users\Matthew\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1960: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
C:\Users\Matthew\Anaconda3\lib\site-packages\numpy\core\_methods.py:122: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)
C:\Users\Matthew\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1465: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(trans.var(axis=0))
C:\Users\Matthew\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1959: RuntimeWarning: invalid value encountered in subtract
  tmp1 = (x - w) * (fx - fv)
C:\Users\Matthew\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1960: RuntimeWarning: invalid value encountered in subtract
  tmp2 = (x - v) * (fx - fw)
C:\Users\M

Apple
Facebook
Google


C:\Users\Matthew\Anaconda3\lib\site-packages\numpy\core\_methods.py:122: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)
C:\Users\Matthew\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1465: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(trans.var(axis=0))
C:\Users\Matthew\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1959: RuntimeWarning: invalid value encountered in subtract
  tmp1 = (x - w) * (fx - fv)
C:\Users\Matthew\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1960: RuntimeWarning: invalid value encountered in subtract
  tmp2 = (x - v) * (fx - fw)
C:\Users\Matthew\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1963: RuntimeWarning: invalid value encountered in greater
  if (tmp2 > 0.0):
C:\Users\Matthew\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1969: RuntimeWarning: invalid value encountered in greater
  if ((p > tmp2 * (a - x)) and (p < tmp2 * (b - x

#### new_X_train

In [18]:
new_X_train = pd.concat(trfm_com_df_lst, axis=0)
print(new_X_train.shape)
new_X_train.head()

(216, 21)


Accounts Receivable  Inventory  Total current assets  \
Year Quarter                                                         
2008 Q2                  1.000000   1.000000              1.000000   
     Q4                  1.046352   1.018178              1.059731   
2009 Q1                  1.000425   1.030468              1.036059   
     Q3                  1.033905   1.089016              1.123494   
     Q4                  1.086813   1.053891              1.113165   

              Total current liabilities  Total liabilities  Common Stock  \
Year Quarter                                                               
2008 Q2                        1.000000           1.001848           1.0   
     Q4                        1.036248           1.066570           1.0   
2009 Q1                        1.014677           1.000000           1.0   
     Q3                        1.087624           1.071278           1.0   
     Q4                        1.099344           1.083033           1.0   

              New Deferred Revenue  New Property and Equipment  \
Year Quarter                                                     
2008 Q2                   1.199735                    1.025731   
     Q4                   1.173824                    1.000000   
2009 Q1                   1.098298                    1.071467   
     Q3                   1.075862                    1.101904   
     Q4                   1.131224                    1.065460   

              Total Non-Current Assets  Total_Assets  Non-Current Liabilities  \
Year Quarter                                                                    
2008 Q2                       1.272701      1.000000                 1.060080   
     Q4                       1.000000      1.007041                 1.180042   
2009 Q1                       1.277832      1.027393                 1.000000   
     Q3                       1.308189      1.096932                 1.053939   
     Q4                       1.260952      1.075823                 1.064682   

              Total Shareholder's Equity  Accounts Payables  Retained Earning  \
Year Quarter                                                                    
2008 Q2                         1.156798           1.000000          1.388793   
     Q4                         1.000000           1.090823          1.437582   
2009 Q1                         1.224426           1.048964          1.253527   
     Q3                         1.276143           1.139437          1.046902   
     Q4                         1.202285           1.158185          1.310867   

                   OCI  Short_term_investments  current_ratio  quick_ratio  \
Year Quarter                                                                 
2008 Q2       1.009421                1.094787       1.826232     1.771855   
     Q4       1.000000                1.000000       1.892237     1.881477   
2009 Q1       1.314248                1.149511       1.900273     1.838581   
     Q3       1.138366                1.245560       1.903074     1.859403   
     Q4       1.291136                1.131042       1.806162     1.807651   

              debt_ratio  debt_to_equity_ratio  equity_multiplier  
Year Quarter                                                       
2008 Q2         1.258568              1.251105           1.243641  
     Q4         2.000000              2.000000           2.000000  
2009 Q1         1.096685              1.100137           1.095950  
     Q3         1.136933              1.138868           1.133541  
     Q4         1.357605              1.340533           1.332355

### Implement the training transformations onto X_test with trans_para_.


In [20]:
test_df_lst = []
for name in company_list:
    company_df = X_test[X_test['company_name']==name]
    test_df_lst.append(company_df)

In [21]:
test_company_dfs = dict(zip(company_list, test_df_lst))

In [24]:
# Too little data!!!! 
test_company_dfs['Microsoft']['Accounts Receivable']

Year  Quarter
2014  Q1         1.349700e+10
2015  Q3         1.144400e+10
2017  Q1         1.288200e+10
2018  Q1         1.720800e+10
Name: Accounts Receivable, dtype: float64

In [25]:
# X_test_new = scaler.transform(X_test)
X_test_new_list = []

for company, trans_para_ in company_trans_para.items():
    
    transformed_df_list = []
    
    for col, parameters in trans_para_.items():

        power_T = parameters[0][0]

        lmbda = parameters[0][1]

        mean_ = parameters[1]

        std_ = parameters[2]

        s = test_company_dfs[company][col]

        # Power Transformation
        if power_T == 'B-C':
            try:
                result = boxcox(s, lmbda)
                
            except: 
#                 print('Original s \n', s)
                ss = s.copy()
                ss = np.array([num+1 if num == 0 else num for num in s ])
#                 print('New s \n', s)
                result = boxcox(ss, lmbda) 
                
#                 print(company, 'has a negative value.')
#                 s.plot(title=col)
#                 print(s)
#                 result = boxcox(abs(s), lmbda)
                
        elif power_T == 'Y-J':
            result = yeojohnson(s, lmbda)
           
        # Difference Transformation
#         diff_res = difference(result, 4)
        
        # StandardScaler 
#         diff_res = array(diff_res).reshape(len(diff_res), 1)
        result = np.array(result).reshape(len(result), 1)
    
        try:
            standard_res = StandardScaler(with_mean = mean_, 
                                     with_std = std_ ).fit_transform(result)
        except:
            print(company, col)
            print('wrong?')
            print(result)
            print('after difference() and reshape', diff_res)
        # Normalization
        norm_res = MinMaxScaler(feature_range=(1,2)).fit_transform(standard_res)
        norm_res = [ s[0] for s in norm_res]
        
        # Transformed Series
        transformed_df = pd.DataFrame(dict({col: norm_res}), index=s.index)
        transformed_df_list.append(transformed_df)
        
    # Transformed Dataframe, gluing series tgt, per company
    trans_df = pd.concat(transformed_df_list, axis=1)
    
    # Append transformed df to df_list
    X_test_new_list.append(trans_df)
    
    print(f'{company} is transformed and added.')

Amazon is transformed and added.
Netflix is transformed and added.
GM is transformed and added.
Microsoft is transformed and added.
Ford is transformed and added.
Apple is transformed and added.
Facebook is transformed and added.
Google is transformed and added.


In [26]:
new_X_Test = pd.concat(X_test_new_list, axis=0)
print(new_X_Test.shape)
new_X_Test.head()

(54, 21)


Accounts Receivable  Inventory  Total current assets  \
Year Quarter                                                         
2007 Q4                  1.000000   1.000000              1.000000   
2008 Q1                  1.091351   1.046824              1.050432   
     Q3                  1.098041   1.094951              1.085208   
2009 Q2                  1.092618   1.096830              1.142007   
2011 Q3                  1.331258   1.397845              1.402166   

              Total current liabilities  Total liabilities  Common Stock  \
Year Quarter                                                               
2007 Q4                        1.000000           1.000000           1.0   
2008 Q1                        1.088711           1.030803           1.0   
     Q3                        1.056393           1.007153           1.0   
2009 Q2                        1.095042           1.031167           1.0   
2011 Q3                        1.350001           1.286902           2.0   

              New Deferred Revenue  New Property and Equipment  \
Year Quarter                                                     
2007 Q4                   1.000000                    1.000000   
2008 Q1                   1.142676                    1.038140   
     Q3                   1.167494                    1.069325   
2009 Q2                   1.026233                    1.115091   
2011 Q3                   1.063691                    1.361383   

              Total Non-Current Assets  Total_Assets  Non-Current Liabilities  \
Year Quarter                                                                    
2007 Q4                       1.000000      1.000000                 1.127505   
2008 Q1                       1.399274      1.079680                 1.021019   
     Q3                       1.423297      1.108999                 1.029242   
2009 Q2                       1.453183      1.150694                 1.000000   
2011 Q3                       1.537167      1.394488                 1.238229   

              Total Shareholder's Equity  Accounts Payables  Retained Earning  \
Year Quarter                                                                    
2007 Q4                         1.000000           1.000000          1.404015   
2008 Q1                         1.261943           1.007451          1.300261   
     Q3                         1.377144           1.060960          1.232611   
2009 Q2                         1.430942           1.094119          1.000000   
2011 Q3                         1.614969           1.400039          1.395643   

                   OCI  Short_term_investments  current_ratio  quick_ratio  \
Year Quarter                                                                 
2007 Q4       1.000000                1.129524       1.832628     1.818130   
2008 Q1       1.098131                1.000000       1.386436     1.468221   
     Q3       1.346135                1.008739       1.940203     1.827057   
2009 Q2       1.312132                1.206737       2.000000     2.000000   
2011 Q3       1.572810                1.531816       1.822973     1.667030   

              debt_ratio  debt_to_equity_ratio  equity_multiplier  
Year Quarter                                                       
2007 Q4         2.000000              2.000000           2.000000  
2008 Q1         1.364138              1.257786           1.252524  
     Q3         1.056946              1.041453           1.039704  
2009 Q2         1.000000              1.000000           1.000000  
2011 Q3         1.022608              1.016769           1.015999

We will be using the same pipeline and hyperparameters as experiment 1.1. 
<a id='expt2.2train'></a>

[Click](#pipe&Hyper)
here to view again.


## Fit and Tune Model with Cross-Validation

In [32]:
# Create empty dictionary called fitted_models
fitted_models = {}

# Loop through model pipelines, tuning each one and saving it to fitted_models
for name, pipeline in pipelines.items():
    # Create cross-validation object from pipeline and hyperparameters
    model = GridSearchCV(pipeline, hyperparameters[name], cv=10, n_jobs=-1)
    
    # Fit model on X_train, y_train
    model.fit(new_X_train, y_train)
    
    # Store model in fitted_models[name] 
    fitted_models[name] = model
    
    # Print '{name} has been fitted'
    print(name, 'has been fitted.')

C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


l1 has been fitted.


C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


l2 has been fitted.


C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


rf has been fitted.
gb has been fitted.


C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [34]:
for name, model in fitted_models.items():
    pred = model.predict_proba(new_X_Test)
    pred = [p[1] for p in pred]
    
    print( name, roc_auc_score(y_test, pred) )

l1 0.45299145299145305
l2 0.5008547008547009
rf 0.5487179487179488
gb 0.5384615384615384


## Expt 1.2 Conclusion
Our best classifer did not perform better than chance. Whether we transform the features across multiple industies as a whole or transform within each company. 
However, unlike expt 1.1, expt 1.2 did not perform the difference transformation. This may introduce some confounding variables, but since we did not have enough data, it is a limitation of this study.

# Experiment 2.1 - MANOVA F-test
<a href='#top'>Top of Page</a>

<a id='expt3.1'></a>

# Experiment 3.1 - k-means clustering
<a href='#top'>Top of Page</a>

<a id='expt4.1'></a>

# Experiment 4.1 - ML Time Series Analysis
<a href='#top'>Top of Page</a>

<a id='appendix'></a>

# Appendix
<a href='#top'>Top of Page</a>

## Comparing BoxCox and Y-J have unexpected results

Inventory positive outcome for Y-J, Total current assets negative outcome for Y-J but positive outcome for Box-Cox.

In [ ]:
Inventory = X_train.Inventory
# print(Inventory)
new_inventory, lmbda = yeojohnson(Inventory) 
pd.DataFrame(new_inventory).plot()
plt.show()

In [ ]:
newB_TCA = brown_transform('Total current assets', X_train['Total current assets'], 4)
# pd.DataFrame(newB_TCA).plot()

In [ ]:
TCA = X_train['Total current assets']
TCA.plot()
plt.show()
# print(TCA.describe())
# print(TCA)


In [ ]:
TCA = X_train['Total current assets']
# print(TCA)
new_TCA, lmbda = yeojohnson(TCA) 
# print(lmbda)
# pd.DataFrame(new_TCA).plot()
# plt.show()

## Try Brown's method.

#### Power Transform

In [ ]:
from scipy.stats import boxcox
s = tsla_df['Accounts Receivable']
result, lmbda = boxcox(s)

In [ ]:
s.plot()

In [ ]:
pd.Series(result).plot()

#### Difference Transform

In [ ]:
# difference dataset
def difference(data, interval):
    return [data[i] - data[i - interval] for i in range(interval, len(data))]

In [ ]:
diff_res = difference(result, 4)
pd.Series(diff_res).plot()

In [ ]:
sample = difference(power_s, 4)
sample = np.array(sample).reshape(-1, 1) 

In [ ]:
sample_norm = pd.DataFrame(StandardScaler().fit_transform(sample))
sample_norm.plot()

#### StandardScaler 

In [ ]:
from numpy import array
diff_res = [x for x in diff_res]
diff_res = array(diff_res).reshape(len(diff_res), 1)
# print(pd.Series(diff_res[0]).plot())
# plt.show()
standard_res = StandardScaler().fit_transform(diff_res)

In [ ]:
pd.DataFrame(standard_res).plot()
plt.show()

#### Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
norm_res = MinMaxScaler().fit_transform(standard_res)

In [ ]:
pd.DataFrame(norm_res).plot()

## Using Scikit-Learn built-in preprocessing methods, does not yield proper results. 

In [ ]:
# Power Transform
from sklearn.preprocessing import power_transform

# Difference Transform
# Missing

# Standardization
from sklearn.preprocessing import StandardScaler

# Normalization
from sklearn.preprocessing import normalize

In [ ]:
type(s.values)

#### Power Transform

In [ ]:
# Using 'Accounts Receivable' as an example, plot different transformations.
s = tsla_df['Short_term_investments']

power_s = power_transform(s.values.reshape(-1,1), 
                          method='box-cox', 
                          standardize=False)
# print(power_s)

##### Accounts Receivable


In [ ]:
# Original
# Before Power Transformations
print(pd.DataFrame(s).plot())
plt.show()

In [ ]:
# Accounts Receivable
# After Power Transformations - box-cox
print(pd.DataFrame(power_s).plot())
plt.show()

In [ ]:
# Accounts Receivable
# After Power Transformations - yeo-johnson
print(pd.DataFrame(power_s).plot())
plt.show()

##### Short_term_investments

In [ ]:
# Original
# Before Power Transformations
print(pd.DataFrame(s).plot())
plt.show()

In [ ]:
# Short_term_investments
# After Power Transformations - box-cox
 **ERROR - **
ValueError: The Box-Cox transformation can only be applied to strictly positive data

In [ ]:
# Short_term_investments
# After Power Transformations - yeo-johnson
print(pd.DataFrame(power_s).plot())
plt.show()

In [ ]:
power_s = [num[0] for num in power_s]

#### Difference Transform
Same as Brown's


In [ ]:
# difference dataset
def difference(data, interval):
    return [data[i] - data[i - interval] for i in range(interval, len(data))]

In [ ]:
diff_s = difference(power_s, 4)
pd.Series(diff_s).plot()

## Normalization

In [ ]:
norm_s = normalize(diff_s, norm='l1', axis=0)
pd.DataFrame(norm_s).plot()

#### Standardization

In [ ]:
diff_s = [[num] for num in diff_s]

In [ ]:
standard_s = StandardScaler().fit_transform(diff_s)
pd.DataFrame(standard_s).plot()

## Comparison of final products between scikit-learn and Brown's process

In [ ]:
norm_s == norm_res
print(type(norm_s), type(norm_res))
norm_s_df = pd.DataFrame( norm_s)
norm_res_df = pd.DataFrame( norm_res)
norm_s_df.plot()
norm_res_df.plot()

<a href='#top'>Top of Page</a>